# Merging Provided Data with Supplemental Data
The provided data set (`data/k.csv`) contains 8 columns: 
1. `MLBAMID`: player's MLB ID 
1. `PlayerId`: player's FanGraphs ID 
1. `Name`: player's name 
1. `Team`: player's team name (**NOTE**: `"- - -"` if the player played on multiple teams in a season)
1. `Age`: player's age in 2024 season
1. `Season`: season year
1. `TBF`: Total batters faced for this player-season 
1. `K%`: Strikeout percentage for this player-season

The supplemental data set (`data/supplemental-stats.csv`) contains 36 columns:
1. `Rk`: arbitrary sorting rank based on selected column 
1. `Name`: player name
1. `Age`: player age at midnight on June 30th of season year
1. `Tm`: abbreviated team name
1. `IP`: innings pitched
1. `PA`: number of plate appearances for which pitch-by-pitch data exists
         (note that inning-ending baserunning outs are counted as a PA, so these may be larger than batting PAs)
1. `Pit`: number of pitches in the PA
1. `Pit/PA`: pitches per plate appearance
1. `Str`: strikes (includes both pitches in the zone and those swung at out of the zone)
1. `Str%`: strike percentage (strikes / total pitches; intentional balls included)
1. `L/Str`: looking strike percentage (strikes looking / total strikes)
1. `S/Str`: swinging strike percentage (swinging strikes w/o contact / total strikes)
1. `F/Str`: foul ball strike percentage (pitches fouled off / total strikes seen)
1. `I/Str`: ball in play percentage (balls put into play including hr / total strikes)
1. `AS/Str`: swung at strike percentage ((inplay + foul + swinging strikes) / total strikes)
1. `I/Bll`: intentional ball percentage (intentional balls / all balls)
1. `AS/Pit`: percentage of pitches swung at ((inplay + foul + swinging strikes) / (total pitches - intentional balls))
1. `Con`: contact percentage ((foul + inplay strikes) / (inplay + foul + swinging strikes))
1. `1st%`: first pitch strike percentage (percent of play appearances being with 0-1 or with a ball inplay
1. `30%`: 3-0 count seen percentage (3-0 counts / PA)
1. `30c`: 3-0 counts seen
1. `30s`: 3-0 count strikes
1. `02%`: 0-2 count seen percentage (0-2 counts / PA)
1. `02c`: 0-2 counts seen
1. `02s`: 0-2 count strikes
1. `02h`: hits given up on an 0-2 count
1. `L/SO`: strikeouts looking
1. `S/SO`: strikeouts swinging
1. `L/SO%`: strikeout looking percentage (stikeouts looking / all strikeouts) 
1. `3pK`: 3 pitch strikeouts
1. `4pW`: 4 pitch walks
1. `PAu`: Plate appearances for which data is unknown
1. `Pitu`: Pitches for which ball-strike results are unknown
1. `Stru`: Strikes for which detailed results are unknown
1. `Season`: Year of stats

In order to create a data set for model prediction, we need to merge these together and aggregate results per player.
- In the the provided data set, each row represents the _aggregated_ stats of an individual pitcher for a given season, even if they played for multiple teams in that season. Take for example, `'A.J. Puk'`'s `2024` season:


|   MLBAMID |   PlayerId | Name     | Team   |   Age |   Season |   TBF |       K% |
|----------:|-----------:|:---------|:-------|------:|---------:|------:|---------:|
|    640462 |      19343 | A.J. Puk | - - -  |    29 |     2024 |   294 | 0.29932  |
|    640462 |      19343 | A.J. Puk | MIA    |    28 |     2023 |   242 | 0.322314 |
|    640462 |      19343 | A.J. Puk | OAK    |    27 |     2022 |   281 | 0.270463 |

A.J. actually played for 3 teams in `2024`: `TOT`, `MIA`, and `ARI`. While this breakout by team doesn't exist in the provided data set, **it does** in the supplemental data set:

|   Rk | Name     |   Age | Tm   |   IP |   PA |   Pit |   Pit/PA |   Str |   Str% | L/Str   |   S/Str |   F/Str |   I/Str |   AS/Str |   I/Bll |   AS/Pit |   Con |   1st% |   30% |   30c |   30s |   02% |   02c |   02s |   02h |   L/SO |   S/SO |   L/SO% |   3pK |   4pW |   PAu |   Pitu |   Stru |   Season |
|-----:|:---------|------:|:-----|-----:|-----:|------:|---------:|------:|-------:|:--------|--------:|--------:|--------:|---------:|--------:|---------:|------:|-------:|------:|------:|------:|------:|------:|------:|------:|-------:|-------:|--------:|------:|------:|------:|-------:|-------:|---------:|
|  782 | A.J. Puk |    29 | TOT  | 71.1 |  294 |  1173 |     3.99 |   781 |  0.666 | 0.26   |   0.236 |   0.279 |   0.225 |    0.74  |       0 |    0.493 | 0.682 |  0.656 | 0.031 |     9 |     7 | 0.333 |    98 |    63 |     4 |     13 |     75 |   0.148 |    22 |     0 |     0 |      0 |      0 |     2024 |
|  783 | A.J. Puk |    29 | MIA  | 44   |  191 |   756 |     3.96 |   486 |  0.643 | 0.26   |   0.193 |   0.292 |   0.251 |    0.737 |       0 |    0.474 | 0.737 |  0.649 | 0.021 |     4 |     4 | 0.277 |    53 |    34 |     2 |      5 |     40 |   0.111 |    12 |     0 |     0 |      0 |      0 |     2024 |
|  784 | A.J. Puk |    29 | ARI  | 27.1 |  103 |   417 |     4.05 |   295 |  0.707 | 0.254   |   0.305 |   0.258 |   0.183 |    0.746 |       0 |    0.528 | 0.591 |  0.67  | 0.049 |     5 |     3 | 0.437 |    45 |    29 |     2 |      8 |     35 |   0.186 |    10 |     0 |     0 |      0 |      0 |     2024 |

In order to merge these data sets together, we will need to fist aggregate the supplemental data so that each player has exactly one row per year, just like the provided data set. While taking the mean across relevant columns is not the most accurate, it will be used as a "best we can do" since we don't have the details to recalculate the things in percentage form (like `L/Str` - looking strike percentage (strikes looking / total strikes), `S/Str`: swinging strike percentage (swinging strikes w/o contact / total strikes) and `F/Str`: foul ball strike percentage (pitches fouled off / total strikes seen)). Calculations will be performed in places where a re-calculate _can_ take place, for example the sum of `PA` or taking `Pit/PA` after aggregation. The appropriate `mean`, `sum`, and/or recalculation will be taken for all columns to collapse multi-team players into one row within the supplemental data set (**NOTE: The former `'Tm'` column is now the `'Team'` column and matches the notation for the provided data set: `- - - `).

|    |   Rk | Name     |   Age | Team   |    IP |   PA |   Pit |   Pit/PA |   Str |   Str% |   L/Str |    S/Str |    F/Str |    I/Str |   AS/Str |   I/Bll |   AS/Pit |   Con |     1st% |       30% |   30c |   30s |      02% |   02c |   02s |   02h |   L/SO |   S/SO |    L/SO% |   3pK |   4pW |   PAu |   Pitu |   Stru |   Season |
|---:|-----:|:---------|------:|:-------|------:|-----:|------:|---------:|------:|-------:|--------:|---------:|---------:|---------:|---------:|--------:|---------:|------:|---------:|----------:|------:|------:|---------:|------:|------:|------:|-------:|-------:|---------:|------:|------:|------:|-------:|-------:|---------:|
|  0 |  782 | A.J. Puk |    29 | - - -  | 142.2 |  588 |  2346 |   3.9898 |  1562 |  0.672 |   0.259 | 0.244667 | 0.276333 | 0.219667 |    0.741 |       0 | 0.498333 |  0.67 | 0.658333 | 0.0306122 |    18 |    14 | 0.333333 |   196 |   126 |     8 |     26 |    150 | 0.147727 |    44 |     0 |     0 |      0 |      0 |     2024 |

In [1]:
import pandas as pd

from bullpen.data_utils import DATA_DIR

print(f"{DATA_DIR=}")

DATA_DIR=PosixPath('/Users/logan/Desktop/repos/mlb-pitcher-xK/data')


## Load Data
See Notebook: `01a-data-processing-fixing-names`
### NOTE
This is folded into `bullpen.data_utils.load_data()` but shown here to give view into entire data processing pipeline.

In [2]:
provided_data = pd.read_csv(DATA_DIR.joinpath("k.csv"))
supplemental_data = pd.read_csv(DATA_DIR.joinpath("supplemental-stats.csv"))


supplemental_data.Name = supplemental_data.Name.replace(
    {
        "Manny Banuelos": "Manny Bañuelos",
        "Ralph Garza": "Ralph Garza Jr.",
        "Luis Ortiz": "Luis L. Ortiz",
        "Jose Hernandez": "Jose E. Hernandez",
        "Hyeon-jong Yang": "Hyeon-Jong Yang",
        "Adrián Martinez": "Adrián Martínez",
    }
)

provided_data.Name = provided_data.Name.replace(
    {
        "Eduardo Rodriguez": "Eduardo Rodríguez",
        "Jose Alvarez": "José Álvarez",
        "Sandy Alcantara": "Sandy Alcántara",
        "Carlos Martinez": "Carlos Martínez",
        "Phillips Valdez": "Phillips Valdéz",
        "Jovani Moran": "Jovani Morán",
        "Jose Cuas": "José Cuas",
        "Jorge Alcala": "Jorge Alcalá",
        "Jhoan Duran": "Jhoan Durán",
        "Jesus Tinoco": "Jesús Tinoco",
        "Brent Honeywell": "Brent Honeywell Jr.",
        "Adrian Morejon": "Adrián Morejón",
    }
)

In [3]:
def aggregate_player_data(supplemental_data, grouper=None):
    """
    Rollup of players that appeared for multiple teams in a season.

    Since all single team players have a singler row in the dataframe,
    operations can be performed on the entire dataframe (summing and
    averaging one row returns the starting values).

    Parameters
    ----------
    supplemental_data : pandas.DataFrame
        Supplemental data with columns numeric data and grouping columns.
    grouper : list of str
        Columns within the dataframe to use for aggregation grouping.

    Returns:
        A new DataFrame with aggregated data and additional re-calculated columns.
    """
    grouper = ["Name", "Age", "Season"] if grouper is None else grouper
    calc_per_col = {
        "Rk": "first",
        "IP": "sum",
        "PA": "sum",
        "Pit": "sum",
        "Str": "sum",
        "Str%": "mean",
        "L/Str": "mean",
        "S/Str": "mean",
        "F/Str": "mean",
        "I/Str": "mean",
        "AS/Str": "mean",
        "I/Bll": "mean",
        "AS/Pit": "mean",
        "Con": "mean",
        "1st%": "mean",
        "30c": "sum",
        "30s": "sum",
        "02c": "sum",
        "02s": "sum",
        "02h": "sum",
        "L/SO": "sum",
        "S/SO": "sum",
        "3pK": "sum",
        "4pW": "sum",
        "PAu": "sum",
        "Pitu": "sum",
        "Stru": "sum",
    }

    numeric_cols = supplemental_data.set_index(grouper).select_dtypes("number").columns

    aggregated = (
        supplemental_data.groupby(grouper)[numeric_cols].agg(calc_per_col).reset_index()
    )

    # Re-calculate columns that need it post aggregation
    aggregated = aggregated.assign(
        **{"Pit/PA": lambda df_: df_.Pit / df_.PA},
        **{"30%": lambda df_: df_["30c"] / df_.PA},
        **{"02%": lambda df_: df_["02c"] / df_.PA},
        **{"L/SO%": lambda df_: df_["L/SO"] / (df_["L/SO"] + df_["S/SO"])},
    )

    first_cols = ["Rk", "Name", "Age"]
    final_cols = first_cols + [
        col for col in supplemental_data.columns if col not in first_cols + ["Tm"]
    ]

    return aggregated[final_cols]

In [4]:
supplemental_data = aggregate_player_data(supplemental_data)

In [5]:
merged = provided_data.merge(
    supplemental_data, on=["Name", "Season", "Age"], how="left"
).sort_values(["Name", "Season", "Team"]).reset_index(drop=True)
merged

,MLBAMID,PlayerId,Name,Team,Age,Season,TBF,K%,Rk,IP,...,02s,02h,L/SO,S/SO,L/SO%,3pK,4pW,PAu,Pitu,Stru
0,621345,18655,A.J. Minter,ATL,27,2021,221,0.257919,696,52.1,...,44,7,11,46,0.192982,11,4,0,0,0
1,621345,18655,A.J. Minter,ATL,28,2022,271,0.346863,649,70.0,...,50,2,23,71,0.244681,12,0,0,0,0
2,621345,18655,A.J. Minter,ATL,29,2023,260,0.315385,647,64.2,...,40,4,13,69,0.158537,8,1,0,0,0
3,621345,18655,A.J. Minter,ATL,30,2024,134,0.261194,676,34.1,...,20,1,7,28,0.200000,6,3,0,0,0
4,640462,19343,A.J. Puk,OAK,27,2022,281,0.270463,773,66.1,...,48,6,22,54,0.289474,15,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1887,554430,10310,Zack Wheeler,PHI,31,2021,849,0.290931,1107,213.1,...,155,15,70,176,0.284553,50,6,0,0,0
1888,554430,10310,Zack Wheeler,PHI,32,2022,607,0.268534,1037,153.0,...,87,5,39,124,0.239264,27,2,0,0,0
1889,554430,10310,Zack Wheeler,PHI,33,2023,787,0.269377,1023,192.0,...,121,8,41,170,0.194313,43,3,0,0,0
1890,554430,10310,Zack Wheeler,PHI,34,2024,787,0.284625,1043,200.0,...,132,11,71,153,0.316964,39,10,0,0,0


In [6]:
merged.dtypes

MLBAMID       int64
PlayerId      int64
Name         object
Team         object
Age           int64
Season        int64
TBF           int64
K%          float64
Rk            int64
IP          float64
PA            int64
Pit           int64
Pit/PA      float64
Str           int64
Str%        float64
L/Str       float64
S/Str       float64
F/Str       float64
I/Str       float64
AS/Str      float64
I/Bll       float64
AS/Pit      float64
Con         float64
1st%        float64
30%         float64
30c           int64
30s           int64
02%         float64
02c           int64
02s           int64
02h           int64
L/SO          int64
S/SO          int64
L/SO%       float64
3pK           int64
4pW           int64
PAu           int64
Pitu          int64
Stru          int64
dtype: object

In [7]:
merged.describe()

,MLBAMID,PlayerId,Age,Season,TBF,K%,Rk,IP,PA,Pit,...,02s,02h,L/SO,S/SO,L/SO%,3pK,4pW,PAu,Pitu,Stru
count,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,...,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.0,1892.0,1892.0
mean,620024.729915,17062.754757,28.740486,2022.503171,344.900634,0.232229,550.721459,91.267706,387.813425,1508.582981,...,57.073996,5.552326,20.501586,68.552854,0.231630,15.587738,4.869979,0.0,0.0,0.0
std,60807.070136,6094.865238,3.675463,1.118325,189.798191,0.055676,315.605461,55.495581,232.248819,899.605015,...,37.100458,4.463840,14.672108,44.150048,0.080260,10.945189,3.819985,0.0,0.0,0.0
min,425794.000000,1246.000000,20.000000,2021.000000,115.000000,0.091463,1.000000,30.000000,115.000000,451.000000,...,5.000000,0.000000,1.000000,6.000000,0.040000,1.000000,0.000000,0.0,0.0,0.0
25%,594426.250000,13435.000000,26.000000,2022.000000,209.000000,0.193899,283.750000,51.100000,221.000000,860.000000,...,30.000000,2.000000,10.000000,37.000000,0.177688,8.000000,2.000000,0.0,0.0,0.0
50%,641712.000000,17485.000000,28.000000,2023.000000,268.000000,0.228360,554.000000,70.000000,294.500000,1158.000000,...,46.000000,4.000000,16.000000,56.000000,0.224490,12.000000,4.000000,0.0,0.0,0.0
75%,664925.500000,20418.000000,31.000000,2024.000000,459.000000,0.266380,822.250000,123.350000,523.250000,2032.250000,...,74.000000,8.000000,27.000000,90.000000,0.277778,20.000000,7.000000,0.0,0.0,0.0
max,808967.000000,33876.000000,43.000000,2024.000000,886.000000,0.502128,1157.000000,383.300000,1616.000000,6382.000000,...,298.000000,36.000000,112.000000,404.000000,0.618182,96.000000,28.000000,0.0,0.0,0.0


In [8]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1892 entries, 0 to 1891
Data columns (total 39 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MLBAMID   1892 non-null   int64  
 1   PlayerId  1892 non-null   int64  
 2   Name      1892 non-null   object 
 3   Team      1892 non-null   object 
 4   Age       1892 non-null   int64  
 5   Season    1892 non-null   int64  
 6   TBF       1892 non-null   int64  
 7   K%        1892 non-null   float64
 8   Rk        1892 non-null   int64  
 9   IP        1892 non-null   float64
 10  PA        1892 non-null   int64  
 11  Pit       1892 non-null   int64  
 12  Pit/PA    1892 non-null   float64
 13  Str       1892 non-null   int64  
 14  Str%      1892 non-null   float64
 15  L/Str     1892 non-null   float64
 16  S/Str     1892 non-null   float64
 17  F/Str     1892 non-null   float64
 18  I/Str     1892 non-null   float64
 19  AS/Str    1892 non-null   float64
 20  I/Bll     1892 non-null   floa